# Demo of MuSCADeT on a single galaxy with multiple components

In [ ]:
#Import libraries
import astropy.io.fits as pf
import numpy as np
import matplotlib.pyplot as plt
from MuSCADeT import MCA as MC
from MuSCADeT  import colour_subtraction as cs

At this point, the data are loaded. Here, we load a pre-computed mixing matrix A, because the colour components in a single galaxy are mixed and extracting them independently can be challenging. In practice, we used other locations of the same field as this cube where galaxies with different colours can be isolated to precompute A.

In [ ]:
#Loading data
cube = pf.open('Simu_2744/Cube.fits')[0].data
nb,n1,n2 = np.shape(cube)
#Pre-computed mixing matrix
Aprior = pf.open('./Simu_2744/Working_A.fits')[0].data

Setting up the parameters for MuSCADeT and running.

In [ ]:
##param
pca = 'None' #Do not use PCA because A is precomputed
n = 100 #Number of iterations
k = 5 #Threshold

#Run MuSCADeT
S, A = MC.mMCA(cube, Aprior, k,n, mode=pca)


Display of the results. The first plot shows the model and a comparison with the original image. The model is simply AS with A and S as estimated by MuSCADeT.

In [ ]:
# Models as extracted by MuSCADeT for display
model = np.dot(A,S.reshape([A.shape[1], n1*n2])).reshape(cube.shape)

normodel = cs.asinh_norm(model, Q=100, range = 1)
normcube = cs.asinh_norm(cube, Q = 100, range = 1)
normres = cs.asinh_norm(cube-model, Q = 10, range = 1)
plt.figure(figsize = (15, 5))
plt.subplot(131)
plt.title('model')
plt.imshow(normodel)
plt.subplot(132)
plt.title('data')
plt.imshow(normcube)
plt.subplot(133)
plt.title('Residuals')
plt.imshow(normres)
plt.show()


This sequence of plots shows each component, with what the data looks like once the components are removed, i.e. Y-AiSi.

In [ ]:
for i in range(A.shape[1]):
    C = A[:,i, np.newaxis, np.newaxis]*S[np.newaxis,i,:,:]
    normC = cs.asinh_norm(C, Q = 100, range = 1)
    normCres = cs.asinh_norm(cube-C, Q = 50, range = 1)
    plt.figure(figsize = (15, 5))
    plt.subplot(131)
    plt.title('data')
    plt.imshow(normcube)
    plt.subplot(132)
    plt.title('component ' + str(i))
    plt.imshow(normC)
    plt.subplot(133)
    plt.title('data - component ' + str(i))
    plt.imshow(normCres)
    plt.show()